### MeSH Based Query Search

In this notebook, we will discuss how to search documents based on the Disease described in the CVD tree.

In [1]:
import pandas as pd
import json
from neo4j import GraphDatabase
import csv

#### Authentication to access covidgraph.org graph

In [2]:
covid_browser = "https://covid.petesis.com:7473"
covid_url = "bolt://covid.petesis.com:7687"
user = "public"
password = "corona"

#driver = GraphDatabase.driver(uri, auth=(user, password))
driver = GraphDatabase.driver(uri = covid_url,\
                              auth = (user,password))

#### MeSH descriptor to its entity list
- Ex. ```C01.925.782.600.550.200.360: [feline infectious peritonitis]```
- Pandas Dataframe is very convenient for handeling a CSV file specifically for data transformation with ```lambda``` mapping functon.

#### First obtain MeSH terms related to corona

In [3]:
MeSH_corona = pd.read_csv("input/mesh/corona.csv")
MeSH_corona = MeSH_corona.set_index('ID')
MeSH_corona.head()

,name
ID,
C01.925.782.600.550.200,Coronavirus Infections
C01.925.782.600.550.200.325,"Enteritis, Transmissible, of Turkeys"
C01.925.782.600.550.200.360,Feline Infectious Peritonitis
C01.925.782.600.550.200.400,"Gastroenteritis, Transmissible, of Swine"
C01.925.782.600.550.200.750,Severe Acute Respiratory Syndrome


- Implementing ```lambda``` function to map one column to another column

In [4]:
MeSH_corona['phrases'] = MeSH_corona['name'].apply(lambda x: x.lower().strip())

In [5]:
MeSH_corona.head()

,name,phrases
ID,,
C01.925.782.600.550.200,Coronavirus Infections,coronavirus infections
C01.925.782.600.550.200.325,"Enteritis, Transmissible, of Turkeys","enteritis, transmissible, of turkeys"
C01.925.782.600.550.200.360,Feline Infectious Peritonitis,feline infectious peritonitis
C01.925.782.600.550.200.400,"Gastroenteritis, Transmissible, of Swine","gastroenteritis, transmissible, of swine"
C01.925.782.600.550.200.750,Severe Acute Respiratory Syndrome,severe acute respiratory syndrome


In [6]:
MeSH_corona['phrases'] = MeSH_corona['phrases'].apply(lambda x:x.split(','))

In [7]:
MeSH_corona.head()

,name,phrases
ID,,
C01.925.782.600.550.200,Coronavirus Infections,[coronavirus infections]
C01.925.782.600.550.200.325,"Enteritis, Transmissible, of Turkeys","[enteritis, transmissible, of turkeys]"
C01.925.782.600.550.200.360,Feline Infectious Peritonitis,[feline infectious peritonitis]
C01.925.782.600.550.200.400,"Gastroenteritis, Transmissible, of Swine","[gastroenteritis, transmissible, of swine]"
C01.925.782.600.550.200.750,Severe Acute Respiratory Syndrome,[severe acute respiratory syndrome]


#### Obtaining terms related to heart disease

In [8]:
MeSH_heart = pd.read_csv("input/mesh/heart_disease.csv")
MeSH_heart = MeSH_heart.set_index('ID')
MeSH_heart.head()

,name
ID,
C14.280.647,Myocardial Ischemia
C14.280.647.124,Acute Coronary Syndrome
C14.280.647.187,Angina Pectoris
C14.280.647.187.150,"Angina, Unstable"
C14.280.647.187.150.150,"Angina Pectoris, Variant"


- Implementing ```lambda``` function to map one column to another column

In [9]:
MeSH_heart['phrases'] = MeSH_heart['name'].apply(lambda x: x.lower().strip())

In [10]:
MeSH_heart['phrases'] = MeSH_heart['phrases'].apply(lambda x:x.split(','))

In [11]:
MeSH_heart.head()

,name,phrases
ID,,
C14.280.647,Myocardial Ischemia,[myocardial ischemia]
C14.280.647.124,Acute Coronary Syndrome,[acute coronary syndrome]
C14.280.647.187,Angina Pectoris,[angina pectoris]
C14.280.647.187.150,"Angina, Unstable","[angina, unstable]"
C14.280.647.187.150.150,"Angina Pectoris, Variant","[angina pectoris, variant]"


#### Combine both value sets
- Corona virus mesh descriptions and heart disease mesh descriptions

In [12]:
all_ = [(x, y) for x in MeSH_corona['phrases'] for y in MeSH_heart['phrases']]

In [13]:
all_[0:4]

[(['coronavirus infections'], ['myocardial ischemia']),
 (['coronavirus infections'], ['acute coronary syndrome']),
 (['coronavirus infections'], ['angina pectoris']),
 (['coronavirus infections'], ['angina', ' unstable'])]

#### MeSH to Doc Mapping
- Create a dictionary where the key is a MeSH descriptor, and the value is a list of papers (publications) that contains mention of the MeSH terms in its body text
- Each paper is represented as dictionary linking each attribute name in the paper (cord_uid, journal, title, etc.) with its actual information

##### Example of a paper node in the covid graph

In [14]:
paper_query = "MATCH (n:Paper) RETURN n LIMIT 1"
Data = []
with driver.session() as session:
    info = session.run(paper_query)
    for item in info:
        print(item)

<Record n=<Node id=3198 labels={'Paper'} properties={'cord_uid': 'zrmkq3mz', 'cord19-fulltext_hash': '41c7a01f11ed47591d99f45774e43e45aeba0619', 'journal': 'BMC Microbiol', 'publish_time': '2009-08-12', 'source': 'PMC', 'title': 'CAPIH: A Web interface for comparative analyses and visualization of host-HIV protein-protein interactions', '_hash_id': '3c4b2ee1430dc9ac53aca87c0fc0f7eb', 'url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2782265/'}>>


#### Writes data to file

In [17]:
ff = open("data/mesh_queries_corona_heartdisease.json", 'w')

In [18]:
for corona, heart in all_:
    #Builds each part of the query based on the MeSH descriptor entity list
    query = "MATCH (p:Paper)-[:PAPER_HAS_BODYTEXTCOLLECTION]-(:BodyTextCollection)-" \
                                        "[:BODYTEXTCOLLECTION_HAS_BODYTEXT]-(a:BodyText) WHERE ("
    for i in range(len(corona)):
        query += "LOWER(a.text) CONTAINS '" + corona[i] + "' AND "
        for j in range(len(heart)):
            if i == len(corona)-1 and j == len(heart)-1:
                query += "LOWER(a.text) CONTAINS '" + heart[j] + "') RETURN DISTINCT p"
            else:
                query += "LOWER(a.text) CONTAINS '" + heart[j] + "' AND "

    MeSH_result = []
        
    with driver.session() as session:
        info = session.run(query)
        for item in info:
            try:
                node_keys = list((item.values(0)[0]).keys())
                node_values = list((item.values(0)[0]).values())
                paper = {}
                for i in range(len(node_keys)):
                    paper[node_keys[i]] = node_values[i]
                MeSH_result.append(paper)
            except:
                continue
    
    if MeSH_result != []:
        print(MeSH_result)
        json.dump(MeSH_result, ff)

ff.close()

[{'cord_uid': 'na3o33b0', 'cord19-fulltext_hash': '42a44518a00b962207226bf61b4d71a0f596e2a1', 'journal': 'Handbook of Vertebrate Immunology', 'publish_time': '1998-12-31', 'source': 'Elsevier', 'title': 'VIII IMMUNOLOGY OF THE CAT', '_hash_id': '81dd955f403d9dd06bfae47b26417cba', 'url': 'https://doi.org/10.1016/b978-012546401-7/50010-x'}]
[{'cord_uid': 'hh6h50t2', 'cord19-fulltext_hash': 'ee31a491e6d7a5a2446d856b7cde6b3924ed1fef', 'journal': 'Antiviral Research', 'publish_time': '2017-10-31', 'source': 'Elsevier', 'title': 'Celastrol inhibits hepatitis C virus replication by upregulating heme oxygenase-1 via the JNK MAPK/Nrf2 pathway in human hepatoma cells', '_hash_id': '2312ccca24588c3c082e1a6049e1882c', 'url': 'https://doi.org/10.1016/j.antiviral.2017.09.010'}]
[{'cord_uid': '9g25ya0k', 'cord19-fulltext_hash': '9f9ab2a290bab64d316a930a3ccea7eb5af2f861', 'journal': 'JACC: Cardiovascular Interventions', 'publish_time': '2020-04-08', 'source': 'Elsevier', 'title': 'Spontaneous Coronary

[{'cord_uid': 'w49i0xkz', 'cord19-fulltext_hash': '320e1640a5ddfcbed21e1afff08a86dc1b0af5b2', 'journal': 'Journal of the American College of Cardiology', 'publish_time': '2020-03-19', 'source': 'Elsevier', 'title': 'Cardiovascular Considerations for Patients, Health Care Workers, and Health Systems During the Coronavirus Disease 2019 (COVID-19) Pandemic', '_hash_id': '363ae9a93261a772bb405b74354d9cda', 'url': 'https://doi.org/10.1016/j.jacc.2020.03.031'}, {'cord_uid': 'k0siih2g', 'cord19-fulltext_hash': '5bdbbdcf847a04980dd506494edb8f02af3b6f33', 'journal': 'Journal of Cardiothoracic and Vascular Anesthesia', 'publish_time': '2020-03-27', 'source': 'Elsevier', 'title': 'Protecting High-Risk Cardiac Patients During the COVID-19 Outbreak', '_hash_id': '9cc84d7f9d4080d07675955282afc926', 'url': 'https://doi.org/10.1053/j.jvca.2020.03.043'}]


In [19]:
f2 = open("data/mesh_texts_corona_heartdisease.json", 'w')

In [20]:
for corona, heart in all_:
    #Builds each part of the query based on the MeSH descriptor entity list
    query = "MATCH (a:BodyText) WHERE ("
    for i in range(len(corona)):
        query += "LOWER(a.text) CONTAINS '" + corona[i] + "' AND "
        for j in range(len(heart)):
            if i == len(corona)-1 and j == len(heart)-1:
                query += "LOWER(a.text) CONTAINS '" + heart[j] + "') RETURN DISTINCT a"
            else:
                query += "LOWER(a.text) CONTAINS '" + heart[j] + "' AND "

    MeSH_result = []
        
    with driver.session() as session:
        info = session.run(query)
        for item in info:
            try:
                node_keys = list((item.values(0)[0]).keys())
                node_values = list((item.values(0)[0]).values())
                paper = {}
                for i in range(len(node_keys)):
                    if (node_keys[i] == 'text'):
                        paper[node_keys[i]] = node_values[i]
                print(paper)
                MeSH_result.append(paper)
            except:
                continue
    
    if MeSH_result != []:
        print(MeSH_result)
        json.dump(MeSH_result, f2)

f2.close()

{'text': 'Studies of feline neutrophil function have evaluated chemotaxis, phagocytosis, and oxidative burst activity during disease states. Feline neutrophils, similar to equine, porcine, bovine, and canine neutrophils, do not respond chemotactically to N-formyl-methionylleucylphenylalanine (FMLP) (Gray et al., 1986) . Neutrophil and endothelial cell interactions involving integrins and selectins have been well studied in the cat and serve as a model for neutrophil-induced endothelial damage in myocardial ischemia (Ma et al., 1991; Lefer and Ma, 1994; Murohara et al., 1994) . Supernatant from cultured peritoneal exudate cells (PEC) from cats with effusive feline infectious peritonitis (FIP) was chemotactic for peripheral blood neutrophils from healthy cats (Tsuji et al., 1989) . Peripheral blood neutrophils collected from FIP-infected cats also showed reduced chemotaxis to zymosan-activated serum while showing similar chemotactic responses to control neutrophils when exposed to PEC su

{'text': "Aside from anticancer, anti-inflammation, and antibacterial/-virus potential, a variety of clinical trials have established other beneficial bioactivities of phenolics. Ferulic acid alleviated angina pectoris and hypertension and is used as a traditional medicine in China [46] . Additionally, ferulic acid reduces the incidence of type 2-diabetes, cardiovascular disease, and neurodegenerative diseases [34, 45, 67] . Chlorogenic and caffeic acids inhibit the formation of mutagenic and carcinogenic N-nitroso compounds [47] . Chao et al. [60] tested the effect of caffeic and ellagic acid on the suppression of diabetic kidney diseases of the rat. The intake of those phenolic acids for 12 weeks significantly reduced the levels of urinary glycated albumin, sorbitol, and fructose as well as suppression of sorbitol dehydrogenase activity. Flavonoids have also shown powerful bioactivities. For instance, catechins and their derivatives relieved degenerative diseases and brain aging proc

{'text': '\nsSA (abbr.1)E (=sino-atrial; cf. sinus node, RSA)SA (abbr.2)E (=stimulus artifact)SAAB (abbr.)E (=Selected And Amplified Binding site imprint assay)SAAG (abbr.)E (=serum-ascites albumin gradient)SaBA (abbr.)E (=sialic acid-binding adhesin)saber tibia/shinsabelschede tibia N (bij syfilis)Sabin trivalent vaccinevaccin van avirulent poliomyelitisvirusSabinas syndrometrichothiodystrofieSabin-Feldman testkleuring voor het aantonen van Toxoplasma-antilichaamSabouraud’s mediumcultuurbodem met pepton en glucosesaburra(onaangename geur van) voedselresten tussen de gebitselementen, id. E (cf. fetid, fetor)saburralkorrelig, zanderig E (=sabulous, gritty)SAC (abbr.)E (=stretch-activated cation channel; cf. VOCC)sac, air ~alveolus van de longsac, heart ~hartzakje, pericardsac, lacrimal ~traanzaksaccadeserie onwillekeurige oogbewegingensaccadiconregelmatige oogbewegingen betreffendsaccharaseinvertase, sacharaseSaccharomycesgistschimmels N (bepaald geslacht)saccharosesacharosesaccular ane

{'text': "Rationale: Premature infants are particularly susceptible to infections. It is estimated that up to 50% of neonates born at <32 weeks gestation may suffer from clinical sepsis. The reasons for this are complex but immaturity of the immune system with sub-optimal levels of circulating immunoglobulins is at least partly responsible for high mortality and morbidity associated with infection. Another protein, mannose-binding lectin (MBL) has also been shown to be low in premature infants and may be important in defending premature infants from infection. Genetic polymorphisms resulting in deficiencies of MBL have been shown to increase susceptibility to infection especially in children. The aim of the current study is decide the role of MBL in determining the susceptibility and outcome of premature neonates to severe sepsis. Methods: A total of 50 premature neonates have been recruited to the study. Serum levels of MBL were measured by enzyme immunoassay and polymorphisms at codo

{'text': 'Our data demonstrated that the serum IP-10 level was significantly increased in AMI patients prior to PCI compared to healthy controls. Recently accumulated evidence has demonstrated an elevated serum level of IP-10 in various diseases, including inflammatory disease, and shown its utility as a clinically informative marker [26] [27] [28] [29] . For instance, Tang et al. reported that early, enhanced induction of IP-10 is an independent predictor of outcome for severe acute respiratory syndrome (SARS) patients [30] . In myocardial infarction, Frangogiannis et al. found that IP-10 is primarily synthesized by ischemic microvascular endothelial cells and that its enhanced production declined within 24 h after reperfusion in an animal ischemiareperfusion model [10] . Rothenbacher et al. reported positive correlations of serum IP-10 levels with several acute-phase proteins or inflammation-associated cytokines in coronary heart disease patients [9] . Aukrust et al. recently reviewe

...

{'text': 'Many of the susceptibility alleles are common polymorphisms, by convention defined as "variations with an allele frequency of 1.01 (1%)." The relationship of the polymorphism to the specific disorder has been identified mainly by association, linkage, and transmission/disequilibrium studies. One of the earliest susceptibility alleles to be identified was that related to Alzheimer disease (MIM #104310), the APOE4 variant of apolipoprotein E (Cys112Arg [MIM ϩ107741.0016]). 69, 70 In many cases, more than one common disorder is related to the same polymorphic allele. In such cases, which common disorder results is thought to depend on the rest of the genetic constitution of the individual and particular environmental circumstances. The different disorders related to a particular susceptibility allele sometimes have obvious possible connections, as in the case of different forms of autoimmune disorders. Even in different indi-viduals in the same family, the predisposition conveye

{'text': 'syndrome (AIDS), and poor health systems (consequential on poverty, mismanagement, and corruption) that afflict a large proportion of developing countries across the tropics. Although, in the past decade, lifestyle issues and changes in diet have led to an increase in the number of noncommunicable disease such as hypertension, diabetes, chronic obstructive airways disease, myocardial infarction, and cerebrovascular accidents in resource-poor tropical countries, tropical infectious diseases remain one of the major causes of preventable morbidity and mortality. 5 Tuberculosis, HIV/AIDS, and malaria alone are currently responsible for an estimated 6 million deaths annually. [1] [2] [3] [4] Schistosomiasis is the second most important parasitic disease after malaria, with 200 million people infected and 779 million at risk in more than 70 countries. In addition to these, leishmaniasis, onchocerciasis, filariasis, Chagas disease, African trypanosomiasis, rickettsioses, enteric fev

{'text': 'PTX3 has an ability to bind to certain microbes, including fungi (conidia of Aspergillus fumigatus, Paracoccidoides brasiliensis and zymosan) [61, 62] ; selected Gram-positive and Gram-negative bacteria (e.g. Pseudomonas aeruginosa, Klebsiella pneumoniae, Neisseria meningitides) [63, 64] and unpublished observation; and viruses (influenza virus type A, human and murine cytomegalovirus, murine hepatitis virus) [65] [66] [67] . In vivo, PTX3 plays non-redundant roles in the defense against infections caused by recognized microbes. Ptx3−/− mice are more susceptible to Protection against acute myocardial infarction -Reduces no-reflow area, IL-6 level, neutrophil infiltration and C3 deposition [72] Protection after ischemic stroke -Reduces blood-brain barrier (BBB) damage, and participates in the resolution of edema and glial scar formation [74] Protection against lung injury -Reduces neutrophil infiltration, cell death and fibrin deposition in LPS-induced ALI [75, 81] Protection 

{'text': 'PTX3 regulates inflammatory responses in infectious and sterile in vivo models, including acute kidney injury [58] , severe acute respiratory syndrome or LPS-induced lung injury [59] , atherosclerosis [60] and acute myocardial infarction [49] , through multifaceted molecular mechanisms. A possible molecular mechanism is provided by binding of PTX3 to the adhesion molecule P-selectin through its N-linked glycosidic moiety, that leads to inhibition of leukocyte recruitment at damaged site [61] . In addition, PTX3 binds key elements of the complement cascade and modulates complement activation [62] . By interacting with C1q through its globular head, PTX3 modulates the classical pathway of complement. PTX3 interacts with the members of the lectin pathway ficolin-1, ficolin-2 and MBL and synergistically amplifies a complement-mediated innate response to Aspergillus fumigatus and Candida albicans [63] . As CRP and SAP, PTX3 also interacts with FH and C4BP [62] on the cell surface 

{'text': 'https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6 (12) . The infectivity of COVID-19 is greater than that of influenza, with an estimated R 0 value (the basic reproduction number, representing viral infectivity) of 2.28 (14) . Notably, the death rate associated with COVID-19 is also considerably higher compared with the most recent WHO estimate of seasonal influenza mortality rate of less than 0.1%, and may reach much higher rates in elderly patients, those with comorbidities, and absent efficient intensive care support (13) . While other zoonotic coronaviruses, including the 2002-2003 severe acute respiratory syndrome (SARS) epidemic and the Middle East respiratory syndrome (MERS-CoV), had higher associated case fatality rates of 9.6% and 34.4%, respectively (15), COVID-19 has resulted in many more deaths than both of these prior outbreaks combined, an issue that is in part related to the greater infectivity and higher attack rate of this 